<a href="https://colab.research.google.com/github/mAbbaspour/Hyperparameter-Optimization/blob/main/hyper%20opt%20%26%20median%20without%20pca%20overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import math;

In [5]:
from google.colab import files
uploaded = files.upload()
uploaded2 = files.upload()


Saving lipo(Lables).csv to lipo(Lables) (1).csv


Saving lipo_global_cdf_rdkit.csv to lipo_global_cdf_rdkit (1).csv


In [6]:
import random
random.seed(1)
import pandas as pd
import io
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv(io.BytesIO(uploaded2['lipo_global_cdf_rdkit.csv']))
dataY = pd.read_csv(io.BytesIO(uploaded['lipo(Lables).csv']))
print(data[data.isnull().any(axis=1)])

      ('RDKit2DNormalized_calculated', <class 'bool'>)  \
1561                                              True   

      ('BalabanJ', <class 'numpy.float64'>)  \
1561                                  0.895   

      ('BertzCT', <class 'numpy.float64'>)  ('Chi0', <class 'numpy.float64'>)  \
1561                              0.230752                           0.031331   

      ('Chi0n', <class 'numpy.float64'>)  ('Chi0v', <class 'numpy.float64'>)  \
1561                            0.032387                            0.061726   

      ('Chi1', <class 'numpy.float64'>)  ('Chi1n', <class 'numpy.float64'>)  \
1561                           0.050799                            0.045963   

      ('Chi1v', <class 'numpy.float64'>)  ('Chi2n', <class 'numpy.float64'>)  \
1561                            0.116251                             0.04344   

      ...  ('fr_sulfonamd', <class 'numpy.float64'>)  \
1561  ...                               1.593061e-17   

      ('fr_sulfone', <class 'nu

In [8]:
data = data.drop(1561)
dataY = dataY.drop(1561)


In [9]:
arrayY = dataY.values
Y = arrayY[0: ,1 ]
array = data.values
X = array[0:,1:]

In [10]:
X_train, X_test ,Y_train , Y_test  = train_test_split(X, Y ,test_size=0.2)
scaler_data  = preprocessing.MinMaxScaler()
X_train = scaler_data.fit_transform(X_train)
X_test =  scaler_data.transform(X_test)
scaler_labels = preprocessing.MinMaxScaler()
Y_train = scaler_labels.fit_transform(Y_train.reshape(-1, 1))
Y_test = scaler_labels.transform(Y_test.reshape(-1, 1))


In [11]:
print(X_test.shape)
print(Y_test.shape)
print(X_train.shape)
print(Y_train.shape)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

(840, 200)
(840, 1)
(3359, 200)
(3359, 1)


In [12]:
Y_train_new =  Y_train.astype(np.float)
Y_test_new = Y_test.astype(np.float)
print(Y_train_new.shape)
print(Y_test_new.shape)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tensor_x_train = torch.tensor(X_train, dtype=torch.float).to(device) 
tensor_x_test = torch.tensor(X_test, dtype=torch.float).to(device) 

tensor_y_train = torch.tensor(Y_train_new , dtype=torch.float).to(device)
tensor_y_test= torch.tensor(Y_test_new, dtype=torch.float).to(device)


(3359, 1)
(840, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [13]:
from torch.utils.data import TensorDataset
trainset = TensorDataset(tensor_x_train, tensor_y_train) 
testset = TensorDataset(tensor_x_test,tensor_y_test)

In [14]:
%%capture
try:
    import ray
except:
    !pip install -U ray
    import ray

In [15]:
from functools import partial
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torchsummary import summary

from ray import tune
# a = {"a":tune.quniform(50, 200, 10) , "b": 5}
# print(type(a.get("a")))
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [16]:
def load_data(data_dir=None):
    return trainset, testset

In [17]:
# # Net().__init__(self , confing) 
#     config = {
#         "act1 ": tune.choice(["relu","tanh","selu"]),
#         "act2" : tune.choice(["relu","tanh","selu"]),
#         "act3" : tune.choice(["relu","tanh","selu"]),
#         "lr": tune.quniform(0.0005, 0.001, 0.0001),
#         "batch_size": tune.choice([8, 16, 32]),
#         "hidden_dim1" : tune.quniform(50, 200, 10),
#         "hidden_dim2" : tune.quniform(50, 200, 10),
#         "hidden_dim3" : tune.quniform(50, 200, 10),
#     }
#   self = { "confing": confing , "hd":}
class Net(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.hidden_dim1 = int(self.config.get("hidden_dim1", 100))
        self.hidden_dim2 = int(self.config.get("hidden_dim2", 100))
        self.hidden_dim3 = int(self.config.get("hidden_dim3", 100))

        self.act1 = self.config.get("act1", "relu") 
        self.act2 = self.config.get("act2", "relu") 
        self.act3 = self.config.get("act3", "relu")

        self.linear1 = nn.Linear(200, self.hidden_dim1)
        self.linear2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.linear3 = nn.Linear(self.hidden_dim2, self.hidden_dim3)
        self.linear4 = nn.Linear(self.hidden_dim3, 1)
    
    @staticmethod
    def activation_func(act_str):
        if act_str=="tanh":
            return eval("torch."+act_str)
        elif act_str=="selu" or act_str=="relu":   
            return eval("torch.nn.functional."+act_str)

    def forward(self, x):
        output = self.linear1(x)
        output = self.activation_func(self.act1)(output)
        output = self.linear2(output)
        output = self.activation_func(self.act2)(output)
        output = self.linear3(output)
        output = self.activation_func(self.act3)(output)
        output = self.linear4(output)        
        return output

In [18]:
model = Net({})

In [19]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [20]:
summary(model, (1,tensor_x_train.shape[1]))

count_parameters(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]          20,100
            Linear-2               [-1, 1, 100]          10,100
            Linear-3               [-1, 1, 100]          10,100
            Linear-4                 [-1, 1, 1]             101
Total params: 40,401
Trainable params: 40,401
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.15
Estimated Total Size (MB): 0.16
----------------------------------------------------------------
+----------------+------------+
|    Modules     | Parameters |
+----------------+------------+
| linear1.weight |   20000    |
|  linear1.bias  |    100     |
| linear2.weight |   10000    |
|  linear2.bias  |    100     |
| linear3.weight |   10000    |
|  linear3.bias  |    100     |
| linear4.we

40401

In [21]:
def trainable_func(config, checkpoint_dir=None, data_dir=None, epochs=10):

    net = Net(config)

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    '''
    Define a loss function
    '''
    ## Classification
    # criterion = nn.CrossEntropyLoss()

    ## Regression
    criterion = nn.MSELoss(reduction='sum')

    # Define an optimizer 
    optimizer = optim.Adam(net.parameters(), lr=config.get("lr",0.0003))

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # Load data
    trainset, testset = load_data(data_dir)

    # Split the dataset into training and validation sets
    train_size = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [train_size, len(trainset) - train_size])

    # Define data loaders (which combines a dataset and a sampler, and provides an iterable over the given dataset)
    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config.get("batch_size",32)),
        shuffle=True,
        num_workers=2)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config.get("batch_size",32)),
        shuffle=True,
        num_workers=2)

    for epoch in range(epochs):  # loop over the dataset multiple times
        epoch_train_loss = 0.0
        # epoch_steps = 0
        net.train() # Prepare model for training
        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            '''
            Compute train loss without scaling to print
            ''' 
            # outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
            # labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device)  
            # loss_train = criterion(outputs, labels) 
            # epoch_train_loss += loss_train.detach().item()
        # print("[%d] loss: %.3f" % (epoch + 1, epoch_train_loss / len(train_subset)))

        # Validation loss
        val_loss = 0.0
        net.eval() # Prepare model for evaluation
        for i, data in enumerate(valloader):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)

                # Inverse transform of the labels' scaler
                outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
                labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device) 

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(epoch = epoch, loss=(val_loss / len(val_subset)))
    print("Finished Training")

In [22]:
def test_score(config, net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=int(config.get("batch_size",32)), shuffle=False, num_workers=2)
    
    ## Regression
    criterion = nn.MSELoss(reduction='sum')

    # Test loss
    test_loss = 0.0
    net.eval() # Prepare model for evaluation
    for i, data in enumerate(testloader):
        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)

            # Inverse transform of the labels' scaler
            outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
            labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device) 

            loss = criterion(outputs, labels)
            test_loss += loss.cpu().numpy()

    return test_loss / len(testset)

In [23]:
ray.shutdown()
ray.init() # Here we use ray.init() to evaluate available_resources for Ray
print(ray.available_resources())
ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 

# Start Ray runtime with specific resources (not nessesarily all resources)
# You can change this values based on your machine resources)
ray.init(num_cpus=2, num_gpus=0) 

"""Check Ray Tune is working properly (for trainable class)"""
# from ray.tune.utils import validate_save_restore
# validate_save_restore(Trainable)
# validate_save_restore(Trainable, use_object_store=True)
# print("Success!")
""""""

{'memory': 8000271975.0, 'CPU': 2.0, 'object_store_memory': 4000135987.0, 'node:172.28.0.2': 1.0}


''

In [24]:
%%capture

%pip install hpbandster ConfigSpace
!pip install hpbandster ConfigSpace

In [25]:
ray.shutdown()
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import MedianStoppingRule
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):

    # define data directory here if you want to load data from files
    data_dir = os.path.abspath("./data")
    load_data(data_dir)

    # define the search space of hyperparameters
    config = {
        "act1 ": tune.choice(["relu","tanh","selu"]),
        "act2" : tune.choice(["relu","tanh","selu"]),
        "act3" : tune.choice(["relu","tanh","selu"]),
        "lr": tune.quniform(0.0005, 0.001, 0.0001),
        "batch_size": tune.choice([ 8,16, 32 , 64]),
        "hidden_dim1" : tune.quniform(120, 330, 15),
        "hidden_dim2" : tune.quniform(120, 330, 15),
        "hidden_dim3" : tune.quniform(120, 330, 15),
    }

    # Optuna search algorithm
    from ray.tune.suggest.optuna import OptunaSearch 
    from ray.tune.suggest import ConcurrencyLimiter
    search_alg = HyperOptSearch(
    # space=config,
    metric="loss", #or accuracy, etc.
    mode="min", #or max
    n_initial_points=60,
    # gamma = 0.1,
    # random_state_seed=42,
    # points_to_evaluate=[{"max_norm_val": 2.5, 'GNN_Layers': 3, 'dropout': 0.25, 'dropout1': 0.35, 'dropout2': 0.15, 'lr': 0.001, 'hidden_size': 80, 'readout1_out': 150, 'readout2_out': 100, 'batch_size': 64}],
    )
    search_alg = ConcurrencyLimiter(search_alg, max_concurrent=10)
    training_iteration = 50
    scheduler = MedianStoppingRule(
    time_attr="training_iteration",
    grace_period=10,
    min_samples_required=10,
    hard_stop = True,
    metric="loss", #or accuracy, etc.
    mode="min", #or max
    )
    
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "training_iteration"]
        )
    
    # wrap data loading and training for tuning using `partial` 
    # (note that there exist other methods for this purpose)
    result = tune.run(
        partial(trainable_func, data_dir=data_dir, epochs=max_num_epochs),
        scheduler=scheduler,
        search_alg=search_alg,
        num_samples=num_samples,
        config=config,
        verbose=2,
        checkpoint_score_attr="loss",
        checkpoint_freq=0,
        keep_checkpoints_num=1,
        # checkpoint_at_end=True,
        # reuse_actors=reuse_actors_status,
        progress_reporter=reporter,
        resources_per_trial={"cpu": 0.5, "gpu": gpus_per_trial},
        stop={"training_iteration": max_num_epochs},                
        )

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation score: {}".format(
        best_trial.last_result["loss"]))

    best_trained_model = Net(best_trial.config)
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_score_value = test_score(best_trial.config, best_trained_model, device)
    print("Best trial test set score: {}".format(test_score_value))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=50, max_num_epochs=50, gpus_per_trial=0)


2022-07-26 11:41:58,588	INFO tune.py:874 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2022-07-26 11:42:04,702	INFO logger.py:630 -- pip install "ray[tune]" to see TensorBoard files.
2022-07-26 11:42:04,705	WARNING callback.py:106 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2022-07-26 11:42:05 (running for 00:00:00.76)
Memory usage on this node: 1.4/12.7 GiB
Using MedianStoppingRule: num_stopped=0.
Resources requested: 0.5/2 CPUs, 0/0 GPUs, 0.0/7.45 GiB heap, 0.0/3.73 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_11-42-04
Number of trials: 1/50 (1 RUNNING)
+-------------------------+----------+-------------------+---------+--------+--------+--------------+---------------+---------------+---------------+--------+
| Trial name              | status   | loc               | act1    | act2   | act3   |   batch_size |   hidden_dim1 |   hidden_dim2 |   hidden_dim3 |     lr |
|-------------------------+----------+-------------------+---------+--------+--------+--------------+---------------+---------------+---------------+--------|
| trainable_func_f8aad432 | RUNNING  | 172.28.0.2:127115 | relu    | tanh   | selu   |            8 |           315 |           315 |           300 | 0.0008 |
+-----------------------

(func pid=127260) 2022-07-26 11:43:59,675	INFO trainable.py:589 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/trainable_func_2022-07-26_11-42-04/trainable_func_f8aad432_1_act1=relu,act2=tanh,act3=selu,batch_size=8,hidden_dim1=315.0000,hidden_dim2=315.0000,hidden_dim3=300.000_2022-07-26_11-42-05/checkpoint_tmpce3d79/./
(func pid=127260) 2022-07-26 11:43:59,676	INFO trainable.py:597 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': None, '_time_total': 107.43532252311707, '_episodes_total': None}


Trial trainable_func_2979c9e2 reported epoch=0,loss=0.9492925419429907,should_checkpoint=True with parameters={'act1 ': 'relu', 'act2': 'tanh', 'act3': 'tanh', 'lr': 0.0008, 'batch_size': 64, 'hidden_dim1': 195.0, 'hidden_dim2': 300.0, 'hidden_dim3': 285.0}.
Trial trainable_func_3c810956 reported epoch=0,loss=0.805144162984962,should_checkpoint=True with parameters={'act1 ': 'relu', 'act2': 'selu', 'act3': 'relu', 'lr': 0.001, 'batch_size': 16, 'hidden_dim1': 270.0, 'hidden_dim2': 270.0, 'hidden_dim3': 315.0}.
== Status ==
Current time: 2022-07-26 11:44:03 (running for 00:01:59.13)
Memory usage on this node: 2.2/12.7 GiB
Using MedianStoppingRule: num_stopped=0.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.45 GiB heap, 0.0/3.73 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_11-42-04
Number of trials: 10/50 (6 PAUSED, 4 RUNNING)
+-------------------------+----------+-------------------+---------+--------+--------+--------------+---------------+---------------+

(func pid=127163) 2022-07-26 11:44:04,873	INFO trainable.py:589 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/trainable_func_2022-07-26_11-42-04/trainable_func_fc6c4d12_2_act1=selu,act2=tanh,act3=tanh,batch_size=8,hidden_dim1=135.0000,hidden_dim2=165.0000,hidden_dim3=135.000_2022-07-26_11-42-11/checkpoint_tmp51617c/./
(func pid=127163) 2022-07-26 11:44:04,873	INFO trainable.py:597 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': None, '_time_total': 65.41247034072876, '_episodes_total': None}


...

Trial trainable_func_863c1652 reported epoch=4,loss=0.3186038262424663,should_checkpoint=True with parameters={'act1 ': 'tanh', 'act2': 'selu', 'act3': 'relu', 'lr': 0.001, 'batch_size': 64, 'hidden_dim1': 270.0, 'hidden_dim2': 300.0, 'hidden_dim3': 210.0}.
Trial trainable_func_e5b588bc reported epoch=9,loss=0.34116864125438345,should_checkpoint=True with parameters={'act1 ': 'relu', 'act2': 'tanh', 'act3': 'selu', 'lr': 0.0006000000000000001, 'batch_size': 32, 'hidden_dim1': 165.0, 'hidden_dim2': 255.0, 'hidden_dim3': 150.0}. This trial completed.
== Status ==
Current time: 2022-07-26 11:58:58 (running for 00:16:54.10)
Memory usage on this node: 2.3/12.7 GiB
Using MedianStoppingRule: num_stopped=37.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.45 GiB heap, 0.0/3.73 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_11-42-04
Number of trials: 50/50 (1 PAUSED, 4 RUNNING, 45 TERMINATED)
+-------------------------+------------+-------------------+---------+--------

(func pid=127260) 2022-07-26 11:58:58,873	INFO trainable.py:589 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/trainable_func_2022-07-26_11-42-04/trainable_func_1e2069d2_44_act1=selu,act2=relu,act3=tanh,batch_size=16,hidden_dim1=150.0000,hidden_dim2=225.0000,hidden_dim3=150.0_2022-07-26_11-57-26/checkpoint_tmp9acbc7/./
(func pid=127260) 2022-07-26 11:58:58,873	INFO trainable.py:597 -- Current state after restoring: {'_iteration': 13, '_timesteps_total': None, '_time_total': 50.45646023750305, '_episodes_total': None}


Trial trainable_func_a52738f4 reported epoch=0,loss=0.13710602986975626,should_checkpoint=True with parameters={'act1 ': 'tanh', 'act2': 'tanh', 'act3': 'tanh', 'lr': 0.0005, 'batch_size': 32, 'hidden_dim1': 315.0, 'hidden_dim2': 150.0, 'hidden_dim3': 285.0}.
Trial trainable_func_590416f8 reported epoch=2,loss=0.30018682499151,should_checkpoint=True with parameters={'act1 ': 'tanh', 'act2': 'selu', 'act3': 'selu', 'lr': 0.0005, 'batch_size': 8, 'hidden_dim1': 180.0, 'hidden_dim2': 180.0, 'hidden_dim3': 150.0}.
Trial trainable_func_1e2069d2 reported epoch=0,loss=0.25601273308389705,should_checkpoint=True with parameters={'act1 ': 'selu', 'act2': 'relu', 'act3': 'tanh', 'lr': 0.0007, 'batch_size': 16, 'hidden_dim1': 150.0, 'hidden_dim2': 225.0, 'hidden_dim3': 150.0}.
Trial trainable_func_863c1652 reported epoch=7,loss=0.3301226658632678,should_checkpoint=True with parameters={'act1 ': 'tanh', 'act2': 'selu', 'act3': 'relu', 'lr': 0.001, 'batch_size': 64, 'hidden_dim1': 270.0, 'hidden_dim

2022-07-26 11:59:37,826	INFO tune.py:748 -- Total run time: 1053.37 seconds (1052.95 seconds for the tuning loop).


Trial trainable_func_a52738f4 reported epoch=27,loss=0.19729104626926924,should_checkpoint=True with parameters={'act1 ': 'tanh', 'act2': 'tanh', 'act3': 'tanh', 'lr': 0.0005, 'batch_size': 32, 'hidden_dim1': 315.0, 'hidden_dim2': 150.0, 'hidden_dim3': 285.0}. This trial completed.
== Status ==
Current time: 2022-07-26 11:59:37 (running for 00:17:32.98)
Memory usage on this node: 1.6/12.7 GiB
Using MedianStoppingRule: num_stopped=40.
Resources requested: 0/2 CPUs, 0/0 GPUs, 0.0/7.45 GiB heap, 0.0/3.73 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_11-42-04
Number of trials: 50/50 (50 TERMINATED)
+-------------------------+------------+-------------------+---------+--------+--------+--------------+---------------+---------------+---------------+--------+-----------+----------------------+
| Trial name              | status     | loc               | act1    | act2   | act3   |   batch_size |   hidden_dim1 |   hidden_dim2 |   hidden_dim3 |     lr |      loss |   tr